In [ ]:
from dotenv import load_dotenv
import sys
import os
import json

load_dotenv(override=True)
try:
    del os.environ["DEV_PREFIX"]
    del os.environ["DEV_ENV"]
except:
    pass

sys.path.insert(0, os.path.join(os.curdir, "../chat/src"))
import core.secrets  # noqa

In [ ]:
from IPython.display import display
from typing import Any, Dict, Optional
from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.messages.tool import ToolMessage
from langchain_core.outputs import LLMResult


class DebugHandler(BaseCallbackHandler):
    def on_llm_start(
        self,
        serialized: dict[str, Any],
        prompts: list[str],
        metadata: Optional[dict[str, Any]] = None,
        **kwargs: Dict[str, Any],
    ):
        print("on_llm_start:")
        display({"serialized": serialized, "metadata": metadata, "kwargs": kwargs})

    def on_llm_end(self, response: LLMResult, **kwargs: Dict[str, Any]):
        print("on_llm_end:")
        display({"response": response, "kwargs": kwargs})

    def on_tool_start(
        self, serialized: Dict[str, Any], input_str: str, **kwargs: Dict[str, Any]
    ):
        print("on_tool_start:")
        display({"serialized": serialized, "kwargs": kwargs})

    def on_tool_end(self, output: ToolMessage, **kwargs: Dict[str, Any]):
        print("on_tool_end:")
        display({"output": output, "kwargs": kwargs})

In [ ]:
import agent.search_agent
from agent.search_agent import SearchAgent
from core.setup import chat_model

model = chat_model(
    model="us.anthropic.claude-3-5-sonnet-20241022-v2:0", streaming=False
)
agent = SearchAgent(model=model)
agent.invoke(
    "What works in the collection pertain to Iranian film?",
    ref="abc123",
    callbacks=[DebugHandler()],
    forget=True,
)